#### La 1 ère partie : 

In [2]:
import re
import math
from collections import defaultdict, Counter

class NgramLanguageModel:
    def __init__(self): 
        self.trigram_counts = defaultdict(int)
        self.bigram_counts = defaultdict(int)
        self.word_counts = Counter()
        self.k = 0.01

    def count_words(self, tokenized_sentences):
        """
        Count the frequency of each word in a list of tokenized sentences.
        """
        word_counter = Counter()
        for sentence in tokenized_sentences:
            word_counter.update(sentence)
        return word_counter

    def get_words_with_nplus_frequency(self, tokenized_sentences, count_threshold):
        """
        Get the list of words with at least 'count_threshold' frequency.
        """
        word_counts = self.count_words(tokenized_sentences)
        closed_vocab = [word for word, count in word_counts.items() if count >= count_threshold]
        return closed_vocab
    
    def replace_oov_words_by_unk(self, tokenized_sentences, vocabulary, unknown_token="<unk>"):
        """
        Replace words not in the vocabulary with the given unknown token.
        """
        vocabulary_set = set(vocabulary)
        replaced_tokenized_sentences = []
        for sentence in tokenized_sentences:
            replaced_sentence = [
                token if token in vocabulary_set else unknown_token
                for token in sentence
            ]
            replaced_tokenized_sentences.append(replaced_sentence)
        return replaced_tokenized_sentences

    def prepare_data(self, infile, ngram_size=2):

        # Lire le contenu du fichier
        with open(infile, 'r', encoding='utf-8') as f:
            raw_text = f.read().strip().lower()

        # Séparer le texte par lignes
        raw_lines = raw_text.split('\n')

        # Tokeniser chaque ligne en mots tout en gardant la ponctuation
        tokenized_lines = [re.findall(r'\w+|[.!?]', line) for line in raw_lines]

        # Définir le nombre minimum d'occurrences pour garder un mot
        min_occurrences = 2

        # Obtenir le vocabulaire des mots fréquents
        vocabulary = self.get_words_with_nplus_frequency(tokenized_lines, min_occurrences)

        # Remplacer les mots OOV par <unk>
        tokenized_lines = self.replace_oov_words_by_unk(tokenized_lines, vocabulary)

        # Ajouter des jetons de début et de fin de phrase
        preprocessed_sentences = []
        for tokens in tokenized_lines:
            if ngram_size == 2:
                preprocessed_sentences.append(['<s>'] + tokens + ['</s>'])
            elif ngram_size == 3:
                preprocessed_sentences.append(['<s>', '<s>'] + tokens + ['</s>'])

        # Joindre toutes les phrases pour obtenir le corpus prétraité
        preprocessed_corpus = ' '.join([' '.join(tokens) for tokens in preprocessed_sentences])

        return preprocessed_corpus
    



    def train(self, infile, ngram_size=2):

        
        preprocessed_corpus = self.prepare_data(infile, ngram_size)

        # Tokeniser le corpus en mots
        tokens = preprocessed_corpus.split()

        if ngram_size == 2:
            # Calculer les bigrams
            for i in range(len(tokens) - 1):
                bigram = (tokens[i], tokens[i + 1])
                self.bigram_counts[bigram] += 1
                self.word_counts[tokens[i]] += 1

            # Compter le dernier mot comme précédent pour le bigram final
            self.word_counts[tokens[-1]] += 1

        elif ngram_size == 3:
            # Calculer les trigrams
            for i in range(len(tokens) - 2):
                trigram = (tokens[i], tokens[i + 1], tokens[i + 2])
                self.trigram_counts[trigram] += 1
            
            # Compter le dernier bigram comme précédent pour le trigram final
            self.word_counts[tokens[-2]] += 1
            self.word_counts[tokens[-1]] += 1
        
        
        # Lissage pour les bigrams
        self.bigram_probabilities = defaultdict(float)
        for bigram, count in self.bigram_counts.items():
            word_1 = bigram[0]
            probability = (count + self.k) / (self.word_counts[word_1] + self.k * len(self.word_counts))
            self.bigram_probabilities[bigram] = math.log(probability)
        
        # Lissage pour les trigrams
        self.trigram_probabilities = defaultdict(float)
        for trigram, count in self.trigram_counts.items():
            bigram = (trigram[0], trigram[1])
            probability = (count + self.k) / (self.bigram_counts.get(bigram,0) + self.k * len(self.word_counts))
            self.trigram_probabilities[trigram] = math.log(probability)


    def predict_ngram(self, sentence, ngram_size=2):
        # Préparer la phrase avec la méthode `prepare_data`
        preprocessed_corpus = self.prepare_data(sentence, ngram_size)

        # Tokeniser le corpus prétraité en mots
        tokens = preprocessed_corpus.split()

        # Initialiser la probabilité totale
        total_log_prob = 0.0

        if ngram_size == 2:
            # Calculer les probabilités des bigrams
            for i in range(len(tokens) - 1):
                bigram = (tokens[i], tokens[i + 1])

                # Obtenir la probabilité du bigram
                if bigram in self.bigram_probabilities:
                    # Ajouter la probabilité logarithmique du bigram
                    total_log_prob += self.bigram_probabilities[bigram]
                else:
                    # Si le bigram n'est pas trouvé, appliquer un lissage supplémentaire
                    vocabulary_size = len(self.word_counts)
                    probability = self.k / (self.k * vocabulary_size)
                    total_log_prob += math.log(probability)

        elif ngram_size == 3:
            # Calculer les probabilités des trigrams
            for i in range(len(tokens) - 2):
                trigram = (tokens[i], tokens[i + 1], tokens[i + 2])

                # Obtenir la probabilité du trigram
                if trigram in self.trigram_probabilities:
                    # Ajouter la probabilité logarithmique du trigram
                    total_log_prob += self.trigram_probabilities[trigram]
                else:
                    # Si le trigram n'est pas trouvé, appliquer un lissage supplémentaire
                    vocabulary_size = len(self.word_counts)
                    probability = self.k / (self.k * vocabulary_size)
                    total_log_prob += math.log(probability)

        return total_log_prob
    

    def test_perplexity(self, test_file, ngram_size=2):
        """
        Calcule la perplexité du modèle entraîné sur un corpus de test.
        """

        # Prétraiter le corpus de test
        preprocessed_corpus = self.prepare_data(test_file, ngram_size)

        # Diviser le corpus en phrases
        sentences = preprocessed_corpus.split('</s>')

        # Calculer la probabilité logarithmique totale
        total_log_prob = 0.0
        total_tokens = 0

        # Pour chaque phrase du corpus de test
        for sentence in sentences:
            sentence = sentence.strip()  # Nettoyer les espaces

            if sentence:  # Si la phrase n'est pas vide
                # Calculez la probabilité logarithmique de la phrase
                log_prob = self.predict_ngram(sentence, ngram_size)
                total_log_prob += log_prob
                
                # Compter le nombre total de tokens, y compris le token de fin de phrase
                num_tokens_in_sentence = len(sentence.split()) + 1
                total_tokens += num_tokens_in_sentence

        # Normaliser le total des log probabilities par le nombre total de tokens
        normalized_log_prob = total_log_prob / total_tokens
        
        # Calculer la perplexité : exp(-normalized_log_prob)
        perplexity = math.exp(-normalized_log_prob)

        return perplexity

    

In [10]:
#Testing prepare_data fct 
ngram_model = NgramLanguageModel()

bigram = ngram_model.prepare_data('test_corpus.txt', ngram_size=2)

print("Corpus prétraité avec Bigram:")
print(bigram)

# Utilisez la méthode 'prepare_data' pour prétraiter le corpus comme trigram
trigram = ngram_model.prepare_data('test_corpus.txt', ngram_size=3)

print("Corpus prétraité avec Trigram:")
print( trigram)


Corpus prétraité avec Bigram:
<s> the quick <unk> fox jumps over the lazy dog . </s> <s> the quick <unk> fox jumps over the lazy dog . </s>
Corpus prétraité avec Trigram:
<s> <s> the quick <unk> fox jumps over the lazy dog . </s> <s> <s> the quick <unk> fox jumps over the lazy dog . </s>


In [11]:
# Testing train fct

ngram_model = NgramLanguageModel()

ngram_model.train('test_corpus.txt', ngram_size=2)

# Afficher quelques bigram counts
print("Comptes de bigrams:")
for bigram, count in list(ngram_model.bigram_counts.items()):
    print(f"{bigram}: {count}")




Comptes de bigrams:
('<s>', 'the'): 2
('the', 'quick'): 2
('quick', '<unk>'): 2
('<unk>', 'fox'): 2
('fox', 'jumps'): 2
('jumps', 'over'): 2
('over', 'the'): 2
('the', 'lazy'): 2
('lazy', 'dog'): 2
('dog', '.'): 2
('.', '</s>'): 2
('</s>', '<s>'): 1


In [12]:
# Afficher quelques probabilités de bigrams
print("\nProbabilités de bigrams:")
for bigram, prob in list(ngram_model.bigram_probabilities.items()):
    print(f"{bigram}: {prob:.5f}")


Probabilités de bigrams:
('<s>', 'the'): -0.04855
('the', 'quick'): -0.71529
('quick', '<unk>'): -0.04855
('<unk>', 'fox'): -0.04855
('fox', 'jumps'): -0.04855
('jumps', 'over'): -0.04855
('over', 'the'): -0.04855
('the', 'lazy'): -0.71529
('lazy', 'dog'): -0.04855
('dog', '.'): -0.04855
('.', '</s>'): -0.04855
('</s>', '<s>'): -0.73674


In [14]:
# Entraînez le modèle avec des trigrams
ngram_model.train('', ngram_size=3)

# Afficher quelques trigram counts
print("\nComptes de trigrams:")
for trigram, count in list(ngram_model.trigram_counts.items()):
    print(f"{trigram}: {count}")



Comptes de trigrams:
('<s>', '<s>', 'the'): 2
('<s>', 'the', 'quick'): 2
('the', 'quick', '<unk>'): 2
('quick', '<unk>', 'fox'): 2
('<unk>', 'fox', 'jumps'): 2
('fox', 'jumps', 'over'): 2
('jumps', 'over', 'the'): 2
('over', 'the', 'lazy'): 2
('the', 'lazy', 'dog'): 2
('lazy', 'dog', '.'): 2
('dog', '.', '</s>'): 2
('.', '</s>', '<s>'): 1
('</s>', '<s>', '<s>'): 1


In [15]:
# Afficher quelques probabilités de trigrams
print("\nProbabilités de trigrams:")
for trigram, prob in list(ngram_model.trigram_probabilities.items()):
    print(f"{trigram}: {prob:.5f}")


Probabilités de trigrams:
('<s>', '<s>', 'the'): 2.90541
('<s>', 'the', 'quick'): -0.04855
('the', 'quick', '<unk>'): -0.04855
('quick', '<unk>', 'fox'): -0.04855
('<unk>', 'fox', 'jumps'): -0.04855
('fox', 'jumps', 'over'): -0.04855
('jumps', 'over', 'the'): -0.04855
('over', 'the', 'lazy'): -0.04855
('the', 'lazy', 'dog'): -0.04855
('lazy', 'dog', '.'): -0.04855
('dog', '.', '</s>'): -0.04855
('.', '</s>', '<s>'): -0.73674
('</s>', '<s>', '<s>'): -0.09441


In [20]:

ngram_model = NgramLanguageModel()
ngram_model.train('test_corpus.txt', ngram_size=2)
log_prob = ngram_model.predict_ngram('test_corpus.txt', ngram_size=2)

print(f"Probabilité logarithmique de la phrase '{'test_corpus.txt'}': {log_prob}")


Probabilité logarithmique de la phrase 'test_corpus.txt': -4.471848899889278
